In [10]:
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, select, sift, sample_n,
    sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction)

In [11]:
combine = '/Users/ryandunlap/Desktop/DSI-SF-2/datasets/football_combine/combine.csv'

In [12]:
combine = DplyFrame(pd.read_csv(combine))
combine.head(2)

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
0,8984,2013,Quanterus Smith,DE,6,5.0,250,33.25,10.375,0.0,...,0,5,Western Kentucky,13(146),13,146,Quanterus,Smith,77.0,0
1,9002,2013,Abry Jones,DT,6,3.0,313,35.00,9.750,0.0,...,30,0,NaN,NaN,0,0,Abry,Jones,75.0,0


In [15]:
position = combine >> select(X.Position)
print position[0:5]

  Position
0       DE
1       DT
2       DT
3       FS
4       FS


In [18]:
#using Pandas:
players = combine[['Name','Weight']]
players = players[players.Weight < 200]
players.head()

In [22]:
dply_player = (combine >>
              sift(X.Weight < 200) >>
              select(X.Name, X.Weight))
dply_player.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [32]:
#Take a random sample of 5% and take out the Name and Weight
sample = combine[['Name','Weight']]
sample = sample.sample(frac=0.05)
sample.head(2)

,Name,Weight
3612,Eric Manning,303
1980,Everette Brown,256


In [30]:
dply_sample = (combine >>
              sample_frac(0.05) >>
              select(X.Name, X.Weight))
dply_sample.head(2)

,Name,Weight
1933,Scott Fujita,248
1874,Erik Olson,208


In [37]:
#Group by position and Year
def weight_plus_arms_mean(df):
    summed = df['Weight'] + df['Arms']
    df['weight_arms_mean'] = np.mean(summed)
    return df.loc[:,['weight_arms_mean']].iloc[0]

combine.groupby(['Position','Year']).apply(weight_plus_arms_mean).reset_index().head(3)

,Position,Year,weight_arms_mean
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000


In [44]:
#deplyr_version
weight_arms_mean = (combine >> 
                   group_by(X.Position, X.Year) >>
                   mutate(weight_arms_sum=X.Weight + X.Arms) >>
                   summarize(weight_arms_mean=X.weight_arms_sum.mean()))
weight_arms_mean.head(3)

,Position,Year,weight_arms_mean
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000


In [47]:
combine.sum()

id                   52591150.00
Year                  9280453.00
HeightFeet              26846.00
HeightInches            20310.50
Weight                1136076.00
Arms                    37796.28
Hands                   11142.75
FortyYD                 21561.79
TwentyYD                  364.76
TenYD                     638.90
TwentySS                15324.49
ThreeCone                5930.02
Vertical               133786.50
Broad                  445979.00
Bench                   73252.00
Round                   11127.00
PickRound               52327.00
PickTotal              329584.00
HeightInchesTotal      342462.50
Wonderlic                5662.00
dtype: float64